In [1]:
#!pip install spotipy

In [2]:
import json
import pandas as pd
import sys

import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials


username = "kelliebanzon"
scope = "user-library-read"

oauth_token = util.prompt_for_user_token(
    username, scope, client_id='c2cf260ff28e4694bcb1eb0a29191b74',
    client_secret='83713ae562c04d7e96220f60749a6989', redirect_uri='http://localhost:8080')
client_credentials_manager = SpotifyClientCredentials(
    client_id="c2cf260ff28e4694bcb1eb0a29191b74", client_secret="83713ae562c04d7e96220f60749a6989")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
ratings = pd.read_csv("/home/jovyan/slo-hacks-2019/tracks.csv")
ratings.head()

,rating,track_1,artist_1,album_1,uri_1,track_2,artist_2,album_2,uri_2,is_pop
0,0.0,Palm Dreams,Hayley Kiyoko,Expectations,spotify:track:3qDaH8tgcdzdpxaL7CyW2s,The 1975,The 1975,"I like it when you sleep, for you are so beaut...",spotify:track:4ZEvWpjUBtL9gjVIzB2Wbr,NaN
1,0.0,Let It Be,Hayley Kiyoko,Expectations,spotify:track:181uk7o0ajEG8F1HYrqbxu,The 1975,The 1975,A Brief Inquiry Into Online Relationships,spotify:track:7dvM0LbJ4pu1tDJnCH1Ahg,NaN
2,0.0,TOOTIMETOOTIMETOOTIME,The 1976,A Brief Inquiry Into Online Relationships,spotify:track:33DzKnwuBE6lfOiADwzd5E,How To Draw / Petrichor,The 1976,A Brief Inquiry Into Online Relationships,spotify:track:1bTfkhyNrSmthxL7P5yGDJ,NaN
3,0.0,Vibe,Tove Stryke,NaN,spotify:track:7rXOrI115JVyhBfM2DRKwZ,Rooftop - Acoustic,Clara Mae,Sorry for Writing All the Songs About You,spotify:track:7m34nMTiU0KwhsMSBmUhde,NaN
4,0.0,Good Mourning,Halsey,hopeless fountain kingdom,spotify:track:6Ck8Lq4srTWQP1PXZ9P8aZ,Wait Up,Charlotte Lawrence,NaN,spotify:track:6VM9H9tmexHFNLePOXOVPX,NaN


In [4]:
ratings.loc[:, "id_1"] = ratings.uri_1.str.strip("spotify:track:")
ratings.loc[:, "id_2"] = ratings.uri_2.str.strip("spotify:track:")
ratings.is_pop = ratings.is_pop.fillna(0)
ratings.head()

,rating,track_1,artist_1,album_1,uri_1,track_2,artist_2,album_2,uri_2,is_pop,id_1,id_2
0,0.0,Palm Dreams,Hayley Kiyoko,Expectations,spotify:track:3qDaH8tgcdzdpxaL7CyW2s,The 1975,The 1975,"I like it when you sleep, for you are so beaut...",spotify:track:4ZEvWpjUBtL9gjVIzB2Wbr,0.0,3qDaH8tgcdzdpxaL7CyW2,4ZEvWpjUBtL9gjVIzB2Wb
1,0.0,Let It Be,Hayley Kiyoko,Expectations,spotify:track:181uk7o0ajEG8F1HYrqbxu,The 1975,The 1975,A Brief Inquiry Into Online Relationships,spotify:track:7dvM0LbJ4pu1tDJnCH1Ahg,0.0,181uk7o0ajEG8F1HYrqbxu,7dvM0LbJ4pu1tDJnCH1Ahg
2,0.0,TOOTIMETOOTIMETOOTIME,The 1976,A Brief Inquiry Into Online Relationships,spotify:track:33DzKnwuBE6lfOiADwzd5E,How To Draw / Petrichor,The 1976,A Brief Inquiry Into Online Relationships,spotify:track:1bTfkhyNrSmthxL7P5yGDJ,0.0,33DzKnwuBE6lfOiADwzd5E,1bTfkhyNrSmthxL7P5yGDJ
3,0.0,Vibe,Tove Stryke,NaN,spotify:track:7rXOrI115JVyhBfM2DRKwZ,Rooftop - Acoustic,Clara Mae,Sorry for Writing All the Songs About You,spotify:track:7m34nMTiU0KwhsMSBmUhde,0.0,7rXOrI115JVyhBfM2DRKwZ,7m34nMTiU0KwhsMSBmUhde
4,0.0,Good Mourning,Halsey,hopeless fountain kingdom,spotify:track:6Ck8Lq4srTWQP1PXZ9P8aZ,Wait Up,Charlotte Lawrence,NaN,spotify:track:6VM9H9tmexHFNLePOXOVPX,0.0,6Ck8Lq4srTWQP1PXZ9P8aZ,6VM9H9tmexHFNLePOXOVPX


#### Workflow:
1. Query SPTTrack using the URI. Collect artist.
2. Using the results from 1, query SPTArtist. Collect genres.

In [7]:
num_queries = 29

for i in range(5):
    offset = i*num_queries
    tracks_list = sp.tracks(ratings.loc[offset:offset+num_queries, "uri_1"],market="US")["tracks"]
    artist_ids = [tracks_list[i]["artists"][0]["id"] for i in range(len(tracks_list))]
    artists_list = sp.artists(artist_ids)["artists"]
    ratings.loc[offset:offset+num_queries, "artist_1_genres"] = [artists_list[i]["genres"] for i in range(len(artists_list))]
ratings.tail()

,rating,track_1,artist_1,album_1,uri_1,track_2,artist_2,album_2,uri_2,is_pop,id_1,id_2,artist_1_genres
141,1.0,DREAMSCAPES,Sam Gellaitry,NaN,spotify:track:15PSdxlFWwvivB7HPU8EmU,LONG DISTANCE,Sam Gellaitry,NaN,spotify:track:5vG7iwUFxDI8xSDfya1Ist,1.0,15PSdxlFWwvivB7HPU8EmU,5vG7iwUFxDI8xSDfya1I,"[electronic trap, vapor twitch]"
142,1.0,____45_____,Bon Iver,NaN,spotify:track:14iePHBA4OZzeMp0gtMehM,00000 Million,Bon Iver,NaN,spotify:track:7BdtdBfrIufJOOCdA0CeJC,1.0,14iePHBA4OZzeMp0gtMehM,7BdtdBfrIufJOOCdA0CeJC,"[chamber pop, folk-pop, indie folk, indie pop,..."
143,1.0,21 MOON WATER,Bon Iver,NaN,spotify:track:3mAyjnYIWLeDixdNO6Joz6,8 (circle),Bon Iver,NaN,spotify:track:47IklCMgkgWvI4jpkdrop0,1.0,3mAyjnYIWLeDixdNO6Joz6,47IklCMgkgWvI4jpkdrop0,"[chamber pop, folk-pop, indie folk, indie pop,..."
144,1.0,Circus for a Psycho,Skillet,NaN,spotify:track:3AhEF2J0fy33PdGYEkzqm2,American Noise,Skillet,NaN,spotify:track:1fnoHCcNPvcLx2ZKyTVFg0,1.0,3AhEF2J0fy33PdGYEkzqm2,1fnoHCcNPvcLx2ZKyTVFg0,"[alternative metal, christian alternative rock..."
145,1.0,The Real Me,The Who,NaN,spotify:track:1ltAKvIg9mdJNdEDme7Seq,Quadrophenia,The Who,NaN,spotify:track:6SjQJEUU1Y2ZiadQjm9KnP,1.0,1ltAKvIg9mdJNdEDme7Seq,6SjQJEUU1Y2ZiadQjm9KnP,"[album rock, art rock, british invasion, class..."


In [8]:
num_queries = 29

for i in range(5):
    offset = i*num_queries
    tracks_list = sp.tracks(ratings.loc[offset:offset+num_queries, "uri_2"],market="US")["tracks"]
    artist_ids = [tracks_list[i]["artists"][0]["id"] for i in range(len(tracks_list))]
    artists_list = sp.artists(artist_ids)["artists"]
    ratings.loc[offset:offset+num_queries, "artist_2_genres"] = [artists_list[i]["genres"] for i in range(len(artists_list))]
ratings.head()

,rating,track_1,artist_1,album_1,uri_1,track_2,artist_2,album_2,uri_2,is_pop,id_1,id_2,artist_1_genres,artist_2_genres
0,0.0,Palm Dreams,Hayley Kiyoko,Expectations,spotify:track:3qDaH8tgcdzdpxaL7CyW2s,The 1975,The 1975,"I like it when you sleep, for you are so beaut...",spotify:track:4ZEvWpjUBtL9gjVIzB2Wbr,0.0,3qDaH8tgcdzdpxaL7CyW2,4ZEvWpjUBtL9gjVIzB2Wb,"[dance pop, electropop, indie poptimism, pop, ...","[modern alternative rock, nu gaze, pop, shiver..."
1,0.0,Let It Be,Hayley Kiyoko,Expectations,spotify:track:181uk7o0ajEG8F1HYrqbxu,The 1975,The 1975,A Brief Inquiry Into Online Relationships,spotify:track:7dvM0LbJ4pu1tDJnCH1Ahg,0.0,181uk7o0ajEG8F1HYrqbxu,7dvM0LbJ4pu1tDJnCH1Ahg,"[dance pop, electropop, indie poptimism, pop, ...","[modern alternative rock, nu gaze, pop, shiver..."
2,0.0,TOOTIMETOOTIMETOOTIME,The 1976,A Brief Inquiry Into Online Relationships,spotify:track:33DzKnwuBE6lfOiADwzd5E,How To Draw / Petrichor,The 1976,A Brief Inquiry Into Online Relationships,spotify:track:1bTfkhyNrSmthxL7P5yGDJ,0.0,33DzKnwuBE6lfOiADwzd5E,1bTfkhyNrSmthxL7P5yGDJ,"[modern alternative rock, nu gaze, pop, shiver...","[modern alternative rock, nu gaze, pop, shiver..."
3,0.0,Vibe,Tove Stryke,NaN,spotify:track:7rXOrI115JVyhBfM2DRKwZ,Rooftop - Acoustic,Clara Mae,Sorry for Writing All the Songs About You,spotify:track:7m34nMTiU0KwhsMSBmUhde,0.0,7rXOrI115JVyhBfM2DRKwZ,7m34nMTiU0KwhsMSBmUhde,"[dance pop, electropop, indie poptimism, pop, ...","[indie cafe pop, indie poptimism, pop, swedish..."
4,0.0,Good Mourning,Halsey,hopeless fountain kingdom,spotify:track:6Ck8Lq4srTWQP1PXZ9P8aZ,Wait Up,Charlotte Lawrence,NaN,spotify:track:6VM9H9tmexHFNLePOXOVPX,0.0,6Ck8Lq4srTWQP1PXZ9P8aZ,6VM9H9tmexHFNLePOXOVPX,"[dance pop, electropop, etherpop, indie poptim...","[dance pop, electropop, indie cafe pop, indie ..."


In [23]:
artist_1_genre_dummies = pd.get_dummies(ratings.loc[:, "artist_1_genres"].apply(pd.Series).stack()).sum(level=0).add_prefix("artist_1 ")
artist_2_genre_dummies = pd.get_dummies(ratings.loc[:, "artist_2_genres"].apply(pd.Series).stack()).sum(level=0).add_prefix("artist_2 ")
ratings = pd.concat([ratings, artist_1_genre_dummies, artist_2_genre_dummies], axis=1)
ratings.head()

,rating,track_1,artist_1,album_1,uri_1,track_2,artist_2,album_2,uri_2,is_pop,...,artist_2 swedish pop,artist_2 symphonic rock,artist_2 talent show,artist_2 tropical house,artist_2 urban contemporary,artist_2 vapor twitch,artist_2 video game music,artist_2 viral pop,artist_2 world worship,artist_2 worship
0,0.0,Palm Dreams,Hayley Kiyoko,Expectations,spotify:track:3qDaH8tgcdzdpxaL7CyW2s,The 1975,The 1975,"I like it when you sleep, for you are so beaut...",spotify:track:4ZEvWpjUBtL9gjVIzB2Wbr,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,Let It Be,Hayley Kiyoko,Expectations,spotify:track:181uk7o0ajEG8F1HYrqbxu,The 1975,The 1975,A Brief Inquiry Into Online Relationships,spotify:track:7dvM0LbJ4pu1tDJnCH1Ahg,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,TOOTIMETOOTIMETOOTIME,The 1976,A Brief Inquiry Into Online Relationships,spotify:track:33DzKnwuBE6lfOiADwzd5E,How To Draw / Petrichor,The 1976,A Brief Inquiry Into Online Relationships,spotify:track:1bTfkhyNrSmthxL7P5yGDJ,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,Vibe,Tove Stryke,NaN,spotify:track:7rXOrI115JVyhBfM2DRKwZ,Rooftop - Acoustic,Clara Mae,Sorry for Writing All the Songs About You,spotify:track:7m34nMTiU0KwhsMSBmUhde,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,Good Mourning,Halsey,hopeless fountain kingdom,spotify:track:6Ck8Lq4srTWQP1PXZ9P8aZ,Wait Up,Charlotte Lawrence,NaN,spotify:track:6VM9H9tmexHFNLePOXOVPX,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
ratings.to_csv("slo-hacks-2019/genres.csv")